In [1]:
!pip install -r <(poetry export --without-hashes)

Ignoring colorama: markers 'python_version >= "3.7" and python_full_version < "3.0.0" and platform_system == "Windows" or platform_system == "Windows" and python_version >= "3.7" and python_full_version >= "3.5.0"' don't match your environment


In [2]:
import sys

import os

import om_course_automation

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from om_course_automation import utils as U

In [3]:
gl = U.get_gitlab_client()

In [4]:
GLO = U.GLO

In [5]:
def get_notes(mr: GLO.ProjectMergeRequest):
    return mr.notes.list(
        all=True,
        as_list=True,
        sort="desc",
        order_by="created_at",
    )

In [6]:
from om_course_automation import singletons as SGL

In [7]:
url = \
'https://docs.google.com/spreadsheets/d/e/2PACX-1vRLD3BfidA72r1RknPDlyd37nARn0J64LVkR-fAGb_GT6_1kp4SD1EyxxwhfUtu43a3Ervi_QMc5b98/pub?gid=0&single=true&output=tsv'

In [37]:
data = SGL.hw_facade._get_spreadsheet_tsv(url)

In [9]:
import pandas as pd

In [38]:
df = pd.DataFrame(data.values())

In [39]:
df['base'] = df['базовый трэк сдан'] == 'TRUE'
df['adv'] = df['advanced трэк сдан'] == 'TRUE'

In [40]:
from tqdm.auto import tqdm

In [41]:
import tinydb

In [42]:
from typing import Iterator, Tuple

In [46]:
def _iter_homework_mrs(
    namespace: str = 'python_2_2022',
) -> Iterator[Tuple[GLO.projects.Project, GLO.MergeRequest]]:
    self = SGL.hw_facade
    
    
    namespace_group = self._gl.groups.get(namespace)
    mrs: List[
        GLO.merge_requests.GroupMergeRequest
    ] = namespace_group.mergerequests.list(all=True, state="closed")
    for mr in mrs:
        prj = self._gl.projects.get(mr.project_id)
        if prj.name != 'day_02_homework':
            continue
        username = prj.namespace["name"]
        if not self._users_db.search(tinydb.Query().username == username):
            self.logger.info(
                "Skipping MR id=%r for project %r since there is no user %r in the db",
                mr.id,
                prj.path_with_namespace,
                username,
            )
            continue
        yield prj, prj.mergerequests.get(mr.iid)

In [47]:
stats = []


In [48]:
for prj, mr in tqdm(_iter_homework_mrs()):
    if not(
        mr.closed_by
        and mr.closed_by.get('username') in ('dgronskiy', 'iporyadkov')
    ):
        continue
    
    iterations = 1
    for n in reversed(get_notes(mr)):
        if n.body == 'marked this merge request as **ready**':
            iterations += 1
    
    stats.append({
        'created_at': mr.created_at,
        'iterations': iterations,
        'username': prj.namespace["name"]
    })

0it [00:00, ?it/s]

In [49]:
sdf = pd.DataFrame(stats)

In [50]:
mask = df.task_01 != ''

In [51]:
m = sdf.merge(df[mask].copy(),
          left_on='username',
          right_on='gitlab_username')

In [52]:
m.shape

(43, 23)

In [53]:
m.groupby('iterations')['base'].mean()

iterations
1    0.700000
2    1.000000
3    0.888889
4    1.000000
6    1.000000
Name: base, dtype: float64

In [63]:
deadline = pd.Timestamp('2022-03-15T00:01:00.000Z')

In [64]:
m['days_to_deadline'] = (
    deadline - pd.to_datetime(m.created_at)
).apply(lambda td:td.days)

In [68]:
m.groupby('days_to_deadline').agg({
  'base': ['mean', 'size'],
  'adv': ['mean', 'size'],
})

base            adv     
                      mean size      mean size
days_to_deadline                              
0                 0.000000    2  0.000000    2
1                 0.833333    6  0.500000    6
2                 1.000000    4  0.000000    4
3                 1.000000    3  1.000000    3
4                 1.000000    2  0.000000    2
5                 1.000000    3  0.666667    3
6                 1.000000    4  0.750000    4
7                 1.000000    5  0.600000    5
8                 1.000000    1  1.000000    1
9                 1.000000    2  0.000000    2
10                1.000000    2  1.000000    2
11                0.500000    2  0.500000    2
12                1.000000    2  0.000000    2
13                1.000000    1  1.000000    1
15                1.000000    1  1.000000    1
16                1.000000    2  1.000000    2
17                1.000000    1  1.000000    1

In [69]:
m.groupby('iterations').agg({
  'base': ['mean', 'size'],
  'adv': ['mean', 'size'],
})

base            adv     
                mean size      mean size
iterations                              
1           0.700000   10  0.400000   10
2           1.000000   18  0.611111   18
3           0.888889    9  0.444444    9
4           1.000000    4  0.750000    4
6           1.000000    2  0.500000    2

In [67]:
m.query('username == "maximpankratov"')

,created_at,iterations,username,gitlab_username,reviewer,"базовый трэк, баллов",advanced трэк баллов,базовый трэк сдан,advanced трэк сдан,обязательные задания сданы,...,task_04,task_05,task_06,task_07,task_08,task_09,task_10,base,adv,days_to_deadline
1,2022-03-14T06:41:37.545Z,1,maximpankratov,maximpankratov,dgronskiy,3,2,FALSE,FALSE,1,...,,1,1,1,1,0,0,False,False,0


In [61]:
print(
    df[mask].base.mean(), '\n',
    df[mask].base.value_counts()
)

0.82 
 True     41
False     9
Name: base, dtype: int64


In [62]:
print(
    df[mask].adv.mean(), '\n',
    df[mask].adv.value_counts()
)

0.46 
 False    27
True     23
Name: adv, dtype: int64


In [58]:
df[mask]..agg({
  'base': ['mean', 'size'],
  'adv': ['mean', 'size']
})

SyntaxError: invalid syntax (1989269241.py, line 1)

In [ ]:
m.days_to_deadline

In [ ]:
iterations = 1
for n in reversed(get_notes(mr)):
    print(n.body)
    if n.body == 'marked this merge request as **ready**':
        print(n)
        iterations += 1

In [ ]:
iterations

In [ ]:
get_notes()

In [ ]:
mr.closed_by

In [ ]:
mr.changes()

In [ ]:
for note in get_notes(mrs[0]):
    print(note.body)